In [ ]:
import os

# Hvis Colab åbnes af Apps Script, start koden automatisk
if "COLAB_AUTO_RUN" in os.environ:
    print("🚀 Colab åbnet af Apps Script – starter billedbehandling...")
    process_images()  # Kør billedbehandling automatisk
else:
    print("🟢 Colab åbnet manuelt – ingen automatisk start.")

import cv2
import numpy as np
import os
import io
from googleapiclient.discovery import build
from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseUpload
from PIL import Image, ImageEnhance

def authenticate_drive():
    SERVICE_ACCOUNT_FILE = "/content/earnest-math-450814-u5-f00d1b302a45.json"
  # Opdater med korrekt filnavn
    SCOPES = ['https://www.googleapis.com/auth/drive']

    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    drive_service = build('drive', 'v3', credentials=credentials)
    return drive_service

def download_image(drive_service, file_id):
    """ Henter en fil fra Google Drive og returnerer et PIL-billede """
    request = drive_service.files().get_media(fileId=file_id)
    file = io.BytesIO()
    downloader = io.BytesIO()
    downloader.write(request.execute())
    file.write(downloader.getvalue())
    file.seek(0)
    return Image.open(file)

def apply_colorgrading(image):
    """Anvender colorgrading og konverterer til JPEG-kompatibelt format"""
    enhancer = ImageEnhance.Color(image)
    image = enhancer.enhance(1.2)  # Justér farvemætning

    # Konverter billedet til RGB, hvis det er RGBA (gennemsigtigt)
    if image.mode == "RGBA":
        image = image.convert("RGB")

    return image

def upload_image(drive_service, image, file_name, output_folder_id):
    image_bytes = io.BytesIO()
    image.save(image_bytes, format='JPEG')
    image_bytes.seek(0)

    media = MediaIoBaseUpload(image_bytes, mimetype='image/jpeg')
    file_metadata = {'name': "Colorgraded_" + file_name, 'parents': [output_folder_id]}

    drive_service.files().create(body=file_metadata, media_body=media).execute()
    print(f"✅ Uploadet: Colorgraded_{file_name}")

def move_file_to_trash_folder(drive_service, file_id, trash_folder_id):
    """Flytter en fil til en Trash-mappe i stedet for at slette den"""
    try:
        file = drive_service.files().update(
            fileId=file_id,
            addParents=trash_folder_id,
            removeParents="1jsM9QFLxPwo0QqhW8uStH_vwIBmDHGnR",
            fields="id, parents"
        ).execute()
        print(f"🗑️ Fil {file_id} er nu flyttet til Trash-mappen!")
    except Exception as e:
        print(f"❌ Fejl ved flytning til Trash-mappen: {e}")

def process_images():
    drive_service = authenticate_drive()
    input_folder_id = "1jsM9QFLxPwo0QqhW8uStH_vwIBmDHGnR"  # Erstat med din input-mappes ID
    output_folder_id = "1_OlzzH2wQVtz0Mj5ODq7rukn45pk5loE"  # Erstat med din output-mappes ID
    trash_folder_id = "1EtrLBoHF4dBmqa2sUinaZwFaWprRVSQp"  # Erstat med din Trash-mappes ID

    results = drive_service.files().list(q=f"'{input_folder_id}' in parents",
                                         fields="files(id, name)").execute()
    files = results.get('files', [])

    for file in files:
        file_id = file['id']
        file_name = file['name']

        # Hent billede fra Google Drive
        image = download_image(drive_service, file_id)

        # Anvend colorgrading
        processed_image = apply_colorgrading(image)

        # Upload farvejusteret billede til Google Drive
        upload_image(drive_service, processed_image, file_name, output_folder_id)

        # Flyt det behandlede billede til Trash-mappen
        move_file_to_trash_folder(drive_service, file_id, trash_folder_id)

    print("✅ Alle billeder er behandlet og flyttet til Trash-mappen.")

# Kør scriptet
process_images()


🟢 Colab åbnet manuelt – ingen automatisk start.
✅ Uploadet: Colorgraded_IMG_2801.JPG
🗑️ Fil 1iA7x5BYA61BgVi7L7eX0LT17lKV7vQvn er nu flyttet til Trash-mappen!
✅ Alle billeder er behandlet og flyttet til Trash-mappen.
